In [3]:
import tweepy
import json
import os
import time
import pprint
import requests
import random
import networkx as nx


In [ ]:

path="data_proj/"
pp=pprint.PrettyPrinter()

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

In [ ]:
api_key="x"
api_secret="x"

access_token="x"
access_secret="x"

bearer_token="x"

In [ ]:
auth=tweepy.OAuthHandler(api_key,api_secret)
auth.set_access_token(access_token,access_secret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print("Auth success")


In [ ]:
users=["mizzaro","damiano10","miccighel_","eglu81","KevinRoitero"]
f_ers=[156,785,331,540,103]
f_ing=[331,836,211,621,256]
utenti=[]
for i in range(0,5):
    utente={}
    utente["name"]=users[i]
    utente["followers"]=f_ers[i]
    utente["following"]=f_ing[i]
    utenti.append(utente)
pp.pprint(utenti)

In [ ]:
#1. Scaricate
followers,following={},{}

for utente in utenti:
    followers_utente,following_utente=[],[]
    for item in tweepy.Cursor(
        api.followers,
        screen_name=utente["name"],
        skip_status=True,
        include_user_entities=False
    ).items():
        json_data=item._json
        new_follower={}
        new_follower["id"]=json_data["id"]
        new_follower["name"]=json_data["name"]
        new_follower["screen_name"]=json_data["screen_name"]
        new_follower["location"]=json_data["location"]
        
    followers_utente.append(new_follower)


    followers[utente["name"]]=followers_utente

    for item in tweepy.Cursor(
        api.friends,
        screen_name=utente["name"],
        skip_status=True,
        include_user_entities=False
    ).items():
        json_data=item._json
        new_following={}
        new_following["id"]=json_data["id"]
        new_following["name"]=json_data["name"]
        new_following["screen_name"]=json_data["screen_name"]
        new_following["location"]=json_data["location"]
    
    following_utente.append(new_following)

    following[utente["name"]]=following_utente


In [ ]:
#export to json
serialize_json("data_proj","followers_5_utenti.json",followers)
serialize_json("data_proj","following_5_utenti.json",following)
#manca readjson e istanzia variabili da dati offline

In [ ]:
#2 
random_followers_utenti,followers_of_followers={},{}

for utente in followers:
    #followers={av1:[],av2:[],...}
    #utente="Iron_Man","thorofficial",...
    random_followers_utenti[utente]=random.sample(followers[utente],5)
#result: random_followers_utenti= {"mizzaro":[5 utenti {}],"damiano10":[5 utenti {}],...}

for utente in followers:
    for f in random_followers_utenti[utente]:
        followers_of_followers[utente]=random.sample(random_followers_utenti[utente],10)    

In [ ]:
#3

random_following_utenti,following_of_following={},{}

for utente in following:
    random_following_utenti[utente]=random.sample(following[utente],5)

for utente in following:
    for f in random_following_utenti[utente]:
        following_of_following[utente]=random.sample(random_following_utenti[utente],10)  


#punto 4 già fatto?

In [4]:
#5
grafo=nx.DiGraph()
#come carichiamo i nodi nel grafo? ci sono più dicts di utenti, magari la cui intersezione non è nulla, bisogna sanificare
for user in followers: #works in th
    for f in followers[user]:
        grafo.add_node(f["id"],name=f["name"],screen_name=f["screen_name"],location=f["location"]) #e se location è omessa?

#todo: creare edges tra nodi(struttura del tipo if(nodoA compare in follows di nodoB)then{edge nodoB to nodoA}